In [52]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore

In [53]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'pcsk_3vHe9e_RDSLWgxS8gCigGYt69vTJobeQPNbPyxC3xzGTW3yD4WEWsMimP7AsqwC3iee8Gg')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east-1')

In [54]:
def load_pdf(data):
    if not os.path.exists(data):
        raise FileNotFoundError(f"Directory '{data}' not found. Please create it or check the path.")

    loader = DirectoryLoader(data,
                              glob="*.pdf",
                              loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

# Call the function
extracted_data = load_pdf("data/")


In [55]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [56]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [57]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [58]:
embeddings = download_hugging_face_embeddings()

In [59]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [60]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [61]:
os.environ["PINECONE_API_KEY"] = "pcsk_3vHe9e_RDSLWgxS8gCigGYt69vTJobeQPNbPyxC3xzGTW3yD4WEWsMimP7AsqwC3iee8Gg"

In [65]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medical-chatbot",
    embedding=embeddings
)

In [66]:
query = "What are Allergies"
docs = docsearch.similarity_search(query, k=3)
print("Result", docs)

Result [Document(id='1238465f-0e2e-45aa-a873-eaec2d202ca1', metadata={}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(id='d6fa0f5c-7494-4619-ad14-54712c300501', metadata={}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reaction

In [67]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [68]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [70]:
from langchain.llms import CTransformers

llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)


In [71]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\Farha\AppData\Local\Temp\ipykernel_12136\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  The first case of AIDS was reported in 1981 in Los Angeles, California. The patient was a gay man named Gaétan Dugas, who was infected with HIV through sexual contact with multiple partners.
Response :  The estimated number of people living with HIV/AIDS in the United States as of 2003 was approximately 1.2 million.
Response :  The estimated number of people living with HIV worldwide in 2003 was around 15 million, according to UNAIDS.
